# Fetch and Format Something Something Data-set

The something something data set is not available publically an account is needded to be created and from [Something Something v2 webpage](https://developer.qualcomm.com/software/ai-datasets/something-something). After which the 20 ziped files can be added to this repository and then the code below can be run

In [ ]:
import os
import subprocess
import shutil

import pandas as pd

In [ ]:
os.mkdir("./data/")
os.mkdir("./data/extracted")

In [ ]:
zip_files = [file for file in os.listdir('./') if '.zip' in file]

In [ ]:
for file in zip_files:
    subprocess.call(
        f'unzip {file} -d ./data/extracted',
        shell=True
    )

In [ ]:
os.chdir("./data/")
print(os.listdir())
subprocess.call(
    f'cat ./extracted/20bn-something-something-v2-?? | tar zx',
    shell=True
)
print(f"number of files extracted: {len(os.listdir('20bn-something-something-v2'))}")
os.chdir("../")

In [ ]:
assert len(os.listdir("./data/20bn-something-something-v2/")) == 220847

## Extract target file classes

In [ ]:
file_prefix = "right_left_up_down_1200"
data_folder = "data_" + file_prefix
data_source = "dat"

if "data_" + file_prefix not in os.listdir():
    os.mkdir(data_folder)
    os.mkdir(data_folder + "/train")
    os.mkdir(data_folder + "/test")
    os.mkdir(data_folder + "/validation")
    shutil.copy("indx_df.csv", data_folder)

indx_df = pd.read_csv(f"{data_folder}/indx_df.csv")

full_ids = list(indx_df ["name"])
train_file_ids = list(indx_df[indx_df["type"] == "train"]["name"])
test_file_ids = list(indx_df[indx_df["type"] == "test"]["name"])
validation_file_ids = list(indx_df[indx_df["type"] == "validation"]["name"])

In [ ]:
assert len(full_ids) == 4800

### Convert To Avi Format

In [ ]:
INPUT_FORMAT = 'webm'
OUTPUT_FORMAT = 'avi'

target_dir = "./data/" + OUTPUT_FORMAT + "/"
cat_dir = "./data/20bn-something-something-v2"

if OUTPUT_FORMAT not in os.listdir("./data/"):
    os.mkdir(target_dir)

failed = []
for f in os.listdir(cat_dir):
    f2 = f.replace("." + INPUT_FORMAT, "." + OUTPUT_FORMAT)
    if f2 in full_ids:
        try: 
            if not os.path.exists(target_dir + f2):
                subprocess.call(
                    f'ffmpeg -i {cat_dir}/{f} {target_dir}/{f2}',
                    shell=True
                )
            else:
                print(f"{f2} already exists")
        except:
            failed.append(f"{f}")
    
#     print(f"failed: {str(failed)}")

In [ ]:
len(os.listdir( "data/avi"))

In [ ]:
data_source = "data/avi"
data_target = "data/avi"


def relocate_files_to_ttv_folder(data_source, folder_prefix, type, file_ids):
    current_files = os.listdir(f"{folder_prefix}/{type}/")

    for f_id in file_ids:
        if f_id not in current_files:
            shutil.copy(f"{data_source}/{f_id}", f"{folder_prefix}/{type}/")


relocate_files_to_ttv_folder(data_source, data_folder, "train", train_file_ids)
relocate_files_to_ttv_folder(data_source, data_folder, "test", test_file_ids)
relocate_files_to_ttv_folder(data_source, data_folder, "validation", validation_file_ids)

In [ ]:
len(os.listdir("data_right_left_up_down_1200/train"))
len(os.listdir("data_right_left_up_down_1200/test"))
len(os.listdir("data_right_left_up_down_1200/validation"))

## Validate files

In [ ]:
from validate_files import validate_files_downloaded

validate_files_downloaded(data_folder)

In [ ]:
from validate_files import check_folder_is_working

check_folder_is_working(data_folder, "test")
check_folder_is_working(data_folder, "validation")
check_folder_is_working(data_folder, "train")